In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import vgg16
from collections import namedtuple
import requests
from tqdm import tqdm
import albumentations
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from torchvision import utils as vutils

In [ ]:
work_dir = "./drive/MyDrive/VAE_learning"

if not os.path.isdir(work_dir):
  os.makedirs(work_dir)

In [ ]:
# Helper Classes
# Group Normalization Layer
class GroupNorm(nn.Module):
  def __init__(self, channels):
    super(GroupNorm, self).__init__()
    self.gn = nn.GroupNorm(num_groups = 32, num_channels=channels)

  def forward(self, x):
    return self.gn(x)

#Swish Layer
class Swish(nn.Module):
  def forward(self, x):
    return x*torch.sigmoid(x)

#ResidualBlock
class ResidualBlock(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(ResidualBlock, self).__init__()
    self.in_channels = in_channels
    self.out_channels = out_channels
    self.block = nn.Sequential(
        GroupNorm(in_channels),
        Swish(),
        nn.Conv2d(in_channels, out_channels, 3, 1, 1),
        GroupNorm(out_channels),
        Swish(),
        nn.Conv2d(out_channels, out_channels, 3, 1, 1),
        GroupNorm(out_channels),
        Swish()
    )

    if in_channels != out_channels:
      self.channel_up = nn.Conv2d(in_channels, out_channels, 1, 1 ,0)

  def forward(self, x):
    if self.in_channels != self.out_channels:
      return self.channel_up(x) + self.block(x)

    else:
      return x + self.block(x)

#UpSample
class UpSampleBlock(nn.Module):
  def __init__(self, channels):
    super(UpSampleBlock, self).__init__()
    self.conv = nn.Conv2d(channels, channels, 3, 1 ,1)

  def forward(self, x):
    x = F.interpolate(x, scale_factor=2.0)
    return self.conv(x)

#DownSample
class DownSampleBlock(nn.Module):
  def __init__(self, channels):
    super(DownSampleBlock, self).__init__()
    self.conv = nn.Conv2d(channels, channels, 3, 2 ,0)

  def forward(self, x):
    pad = (0, 1, 0, 1)
    x = F.pad(x, pad, mode="constant", value=0)
    return self.conv(x)

#NonLocalBlock
class NonLocalBlock(nn.Module):
    def __init__(self, channels):
        super(NonLocalBlock, self).__init__()
        self.in_channels = channels

        self.gn = GroupNorm(channels)
        self.q = nn.Conv2d(channels, channels, 1, 1, 0)
        self.k = nn.Conv2d(channels, channels, 1, 1, 0)
        self.v = nn.Conv2d(channels, channels, 1, 1, 0)
        self.proj_out = nn.Conv2d(channels, channels, 1, 1, 0)

    def forward(self, x):
        h_ = self.gn(x)
        q = self.q(h_)
        k = self.k(h_)
        v = self.v(h_)

        b, c, h, w = q.shape

        q = q.reshape(b, c, h*w)
        q = q.permute(0, 2, 1)
        k = k.reshape(b, c, h*w)
        v = v.reshape(b, c, h*w)

        attn = torch.bmm(q, k)
        attn = attn * (int(c)**(-0.5))
        attn = F.softmax(attn, dim=2)
        attn = attn.permute(0, 2, 1)

        A = torch.bmm(v, attn)
        A = A.reshape(b, c, h, w)

        return x + A




In [ ]:
#Encoder

class Encoder(nn.Module):
  def __init__(self, args):
    super(Encoder, self).__init__()
    channels = [128,128,128,256,256,512]
    attn_resolutions = [16]
    num_res_blocks=2
    resolution = 256
    layers=[nn.Conv2d(args.image_channels, channels[0], 3, 1, 1)]
    for i in range(len(channels)-1):
      in_channels = channels[i]
      out_channels = channels[i+1]
      for j in range(num_res_blocks):
        layers.append(ResidualBlock(in_channels, out_channels))
        in_channels = out_channels
        if resolution in attn_resolutions:
          layers.append(NonLocalBlock(in_channels))
      if i != len(channels)-2:
        layers.append(DownSampleBlock(channels[i+1]))
        resolution //= 2
    layers.append(ResidualBlock(channels[-1], channels[-1]))
    layers.append(NonLocalBlock(channels[-1]))
    layers.append(ResidualBlock(channels[-1], channels[-1]))
    layers.append(GroupNorm(channels[-1]))
    layers.append(Swish())
    layers.append(nn.Conv2d(channels[-1], args.latent_dim, 3, 1, 1))
    self.model = nn.Sequential(*layers)

  def forward(self, x):
    return self.model(x)


In [ ]:
#Decoder

class Decoder(nn.Module):
  def __init__(self, args):
    super(Decoder, self).__init__()
    channels = [512,256,256,128,128]
    attn_resolutions=[16]
    num_res_blocks = 3
    resolution = 16
    in_channels = channels[0]

    layers = [nn.Conv2d(args.latent_dim, in_channels, 3, 1, 1),
              ResidualBlock(in_channels, in_channels),
              NonLocalBlock(in_channels),
              ResidualBlock(in_channels, in_channels)]

    for i in range(len(channels)):
      out_channels = channels[i]
      for j in range(num_res_blocks):
        layers.append(ResidualBlock(in_channels, out_channels))
        in_channels = out_channels
        if resolution in attn_resolutions:
          layers.append(NonLocalBlock(in_channels))
      if i != 0:
        layers.append(UpSampleBlock(in_channels))
        resolution*=2

    layers.append(GroupNorm(in_channels))
    layers.append(Swish())
    layers.append(nn.Conv2d(in_channels, args.image_channels, 3, 1 ,1))
    self.model = nn.Sequential(*layers)

  def forward(self,x):
    return self.model(x)

In [ ]:
#Codebook

class Codebook(nn.Module):
  def __init__(self, args):
    super(Codebook, self).__init__()
    self.num_codebook_vectors = args.num_codebook_vectors
    self.latent_dim = args.latent_dim
    self.beta = args.beta

    self.embedding = nn.Embedding(self.num_codebook_vectors, self.latent_dim)
    self.embedding.weight.data.uniform_(-1.0 /self.num_codebook_vectors, -1.0 /self.num_codebook_vectors)

  def forward(self, z):
    z = z.permute(0,2,3,1).contiguous()
    z_flattened = z.view(-1, self.latent_dim)

    d = torch.sum(z_flattened**2, dim=1, keepdim=True) + \
        torch.sum(self.embedding.weight ** 2, dim=1) - \
        2*torch.matmul(z_flattened, self.embedding.weight.t())

    min_encoding_indices = torch.argmin(d, dim=1)
    z_q = self.embedding(min_encoding_indices).view(z.shape)

    loss = torch.mean((z_q.detach() - z)**2 + self.beta + (z_q - z.detach())**2)

    z_q = z -(z_q.detach())

    z_q = z_q.permute(0,3,1,2)

    return z_q, min_encoding_indices, loss




In [ ]:
#VQGAN

class VQGAN(nn.Module):
  def __init__(self, args):
    super(VQGAN,self).__init__()
    self.encoder = Encoder(args).to(device=args.device)
    self.decoder = Decoder(args).to(device=args.device)
    self.codebook = Codebook(args).to(device=args.device)
    self.quant_conv = nn.Conv2d(args.latent_dim, args.latent_dim, kernel_size=1).to(device=args.device)
    self.post_quant_conv = nn.Conv2d(args.latent_dim, args.latent_dim, kernel_size=1).to(device=args.device)

  def forward(self, imgs):
    encoded_images = self.encoder(imgs)
    quant_conv_encoded_images = self.quant_conv(encoded_images)
    codebook_mapping, codebook_indices, q_loss = self.codebook(quant_conv_encoded_images)
    post_quant_conv_mapping = self.post_quant_conv(codebook_mapping)
    decoded_images = self.decoder(post_quant_conv_mapping)

    return decoded_images, codebook_indices, q_loss

  def encode(self, imgs):
    encoded_images = self.encoder(imgs)
    quant_conv_encoded_images = self.quant_conv(encoded_images)
    codebook_mapping, codebook_indices, q_loss = self.codebook(quant_conv_encoded_images)

    return codebook_mapping, codebook_indices, q_loss

  def decode(self, z):
    post_quant_conv_mapping = self.post_quant_conv(z)
    decoded_images = self.decoder(post_quant_conv_mapping)

    return decoded_images

  def calculate_lambda(self, perceptual_loss, gan_loss):
    last_layer = self.decoder.model[-1]
    last_layer_weight = last_layer.weight

    perceptual_loss_grads = torch.autograd.grad(perceptual_loss, last_layer_weight, retain_graph=True)[0]
    gan_loss_grads = torch.autograd.grad(gan_loss, last_layer_weight, retain_graph=True)[0]

    l = torch.norm(perceptual_loss_grads)/(torch.norm(gan_loss_grads)-1e-4)
    l = torch.clamp(l, 0, 1e-4).detach()

    return 0.8 * l

  @staticmethod
  def adopt_weight(disc_factor, i, threshold, value=0.):
      if i < threshold:
          disc_factor = value
      return disc_factor


  def load_checkpoint(self, path):
    self.load_state_dict(torch.load(path, map_location=torch.device("cuda" if torch.cuda.is_available() else 'cpu')))

In [ ]:
#Discriminator

class Discriminator(nn.Module):
    def __init__(self, args, num_filters_last=64, n_layers=3):
        super(Discriminator, self).__init__()

        layers = [nn.Conv2d(args.image_channels, num_filters_last, 4, 2, 1), nn.LeakyReLU(0.2)]
        num_filters_mult = 1

        for i in range(1, n_layers + 1):
            num_filters_mult_last = num_filters_mult
            num_filters_mult = min(2 ** i, 8)
            layers += [
                nn.Conv2d(num_filters_last * num_filters_mult_last, num_filters_last * num_filters_mult, 4,
                          2 if i < n_layers else 1, 1, bias=False),
                nn.BatchNorm2d(num_filters_last * num_filters_mult),
                nn.LeakyReLU(0.2, True)
            ]

        layers.append(nn.Conv2d(num_filters_last * num_filters_mult, 1, 4, 1, 1))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
      return self.model(x)

In [ ]:
#LPIP




URL_MAP = {
    "vgg_lpips": "https://heibox.uni-heidelberg.de/f/607503859c864bc1b30b/?dl=1"
}

CKPT_MAP = {
    "vgg_lpips": "vgg.pth"
}


def download(url, local_path, chunk_size=1024):
    os.makedirs(os.path.split(local_path)[0], exist_ok=True)
    with requests.get(url, stream=True) as r:
        total_size = int(r.headers.get("content-length", 0))
        with tqdm(total=total_size, unit="B", unit_scale=True) as pbar:
            with open(local_path, "wb") as f:
                for data in r.iter_content(chunk_size=chunk_size):
                    if data:
                        f.write(data)
                        pbar.update(chunk_size)


def get_ckpt_path(name, root):
    assert name in URL_MAP
    path = os.path.join(root, CKPT_MAP[name])
    if not os.path.exists(path):
        print(f"Downloading {name} model from {URL_MAP[name]} to {path}")
        download(URL_MAP[name], path)
    return path


class LPIPS(nn.Module):
    def __init__(self):
        super(LPIPS, self).__init__()
        self.scaling_layer = ScalingLayer()
        self.channels = [64, 128, 256, 512, 512]
        self.vgg = VGG16()
        self.lins = nn.ModuleList([
            NetLinLayer(self.channels[0]),
            NetLinLayer(self.channels[1]),
            NetLinLayer(self.channels[2]),
            NetLinLayer(self.channels[3]),
            NetLinLayer(self.channels[4])
        ])

        self.load_from_pretrained()

        for param in self.parameters():
            param.requires_grad = False

    def load_from_pretrained(self, name="vgg_lpips"):
        ckpt = get_ckpt_path(name, "./drive/MyDrive/VAE_learning")
        self.load_state_dict(torch.load(ckpt, map_location=torch.device("cuda:0")), strict=False)

    def forward(self, real_x, fake_x):
        features_real = self.vgg(self.scaling_layer(real_x))
        features_fake = self.vgg(self.scaling_layer(fake_x))
        diffs = {}

        for i in range(len(self.channels)):
            diffs[i] = (norm_tensor(features_real[i]) - norm_tensor(features_fake[i])) ** 2

        return sum([spatial_average(self.lins[i].model(diffs[i])) for i in range(len(self.channels))])



class ScalingLayer(nn.Module):
    def __init__(self):
        super(ScalingLayer, self).__init__()
        self.register_buffer("shift", torch.Tensor([-.030, -.088, -.188])[None, :, None, None])
        self.register_buffer("scale", torch.Tensor([.458, .448, .450])[None, :, None, None])

    def forward(self, x):
        return (x - self.shift) / self.scale

class NetLinLayer(nn.Module):
    def __init__(self, in_channels, out_channels=1):
        super(NetLinLayer, self).__init__()
        self.model = nn.Sequential(
            nn.Dropout(),
            nn.Conv2d(in_channels, out_channels, 1, 1, 0, bias=False)
        )


class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        vgg_pretrained_features = vgg16(pretrained=True).features
        slices = [vgg_pretrained_features[i] for i in range(30)]
        self.slice1 = nn.Sequential(*slices[0:4])
        self.slice2 = nn.Sequential(*slices[4:9])
        self.slice3 = nn.Sequential(*slices[9:16])
        self.slice4 = nn.Sequential(*slices[16:23])
        self.slice5 = nn.Sequential(*slices[23:30])

        for param in self.parameters():
            param.requires_grad = False

    def forward(self, x):
        h = self.slice1(x)
        h_relu1 = h
        h = self.slice2(h)
        h_relu2 = h
        h = self.slice3(h)
        h_relu3 = h
        h = self.slice4(h)
        h_relu4 = h
        h = self.slice5(h)
        h_relu5 = h
        vgg_outputs = namedtuple("VGGOutputs", ['relu1_2', 'relu2_2', 'relu3_3', 'relu4_3', 'relu5_3'])
        return vgg_outputs(h_relu1, h_relu2, h_relu3, h_relu4, h_relu5)


def norm_tensor(x):
    """
    Normalize images by their length to make them unit vector?
    :param x: batch of images
    :return: normalized batch of images
    """
    norm_factor = torch.sqrt(torch.sum(x**2, dim=1, keepdim=True))
    return x / (norm_factor + 1e-10)


def spatial_average(x):
    """
     imgs have: batch_size x channels x width x height --> average over width and height channel
    :param x: batch of images
    :return: averaged images along width and height
    """
    return x.mean([2, 3], keepdim=True)

In [ ]:
#Utils



# --------------------------------------------- #
#                  Data Utils
# --------------------------------------------- #

class ImagePaths(Dataset):
    def __init__(self, path, size=None):
        self.size = size

        layout = BIDSLayout(os.join)

        self.images = [os.path.join(path, file) for file in os.listdir(path)]
        self._length = len(self.images)

        self.rescaler = albumentations.SmallestMaxSize(max_size=self.size)
        self.cropper = albumentations.CenterCrop(height=self.size, width=self.size)
        self.preprocessor = albumentations.Compose([self.rescaler, self.cropper])

    def __len__(self):
        return self._length

    def preprocess_image(self, image_path):
        image = Image.open(image_path)
        if not image.mode == "RGB":
            image = image.convert("RGB")
        image = np.array(image).astype(np.uint8)
        image = self.preprocessor(image=image)["image"]
        image = (image / 127.5 - 1.0).astype(np.float32)
        image = image.transpose(2, 0, 1)
        return image

    def __getitem__(self, i):
        example = self.preprocess_image(self.images[i])
        return example


def load_data(args):
    train_data = ImagePaths(args.dataset_path, size=256)

    train_loader = DataLoader(train_data, batch_size=args.batch_size, shuffle=False)
    return train_loader


# --------------------------------------------- #
#                  Module Utils
#            for Encoder, Decoder etc.
# --------------------------------------------- #

def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.normal_(m.weight.data, 0.0, 0.02).to(device = "cuda:0")
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.normal_(m.weight.data, 1.0, 0.02).to(device = "cuda:0")
        nn.init.constant_(m.bias.data, 0).to(device = "cuda:0")


def plot_images(images):
    x = images["input"]
    reconstruction = images["rec"]
    half_sample = images["half_sample"]
    full_sample = images["full_sample"]

    fig, axarr = plt.subplots(1, 4)
    axarr[0].imshow(x.cuda().detach().numpy()[0].transpose(1, 2, 0))
    axarr[1].imshow(reconstruction.cuda().detach().numpy()[0].transpose(1, 2, 0))
    axarr[2].imshow(half_sample.cuda().detach().numpy()[0].transpose(1, 2, 0))
    axarr[3].imshow(full_sample.cuda().detach().numpy()[0].transpose(1, 2, 0))
    plt.show()

In [ ]:
# Training VQGAN

class TrainVQGAN:
  def __init__(self, args):
    self.work_dir = args.work_dir
    self.results_dir = os.path.join(self.work_dir, "results")
    self.checkpoint_dir = os.path.join(self.work_dir, "checkpoints")

    self.vqgan = VQGAN(args).to(device=torch.device(args.device))
    self.discriminator = Discriminator(args).to(device=torch.device(args.device))
    self.discriminator.apply(weights_init)
    self.discriminator.to(device=torch.device(args.device))
    self.perceptual_loss = LPIPS().eval().to(device=torch.device(args.device))
    self.opt_vq, self.opt_disc = self.configure_optimizers(args)

    self.prepare_training()

    self.train(args)

  def configure_optimizers(self, args):
    lr = args.learning_rate
    opt_vq = torch.optim.Adam(
        list(self.vqgan.encoder.parameters()) + \
        list(self.vqgan.decoder.parameters()) + \
        list(self.vqgan.codebook.parameters()) + \
        list(self.vqgan.quant_conv.parameters()) + \
        list(self.vqgan.post_quant_conv.parameters()),
        lr=lr, eps = 1e-8, betas=(args.beta1, args.beta2)
    )
    opt_disc = torch.optim.Adam(self.discriminator.parameters(), lr=lr, eps=1e-8, betas=(args.beta1, args.beta2))

    return opt_vq, opt_disc

  @staticmethod
  def prepare_training():
    os.makedirs(os.path.join(work_dir, "results"), exist_ok=True)
    os.makedirs(os.path.join(work_dir, "checkpoints"), exist_ok=True)

  def train(self, args):
    train_dataset = load_data(args)
    steps_per_epoch = len(train_dataset)
    for epoch in range(args.epochs):
        with tqdm(range(len(train_dataset))) as pbar:
            for i, imgs in zip(pbar, train_dataset):
                imgs = imgs.to(device=args.device)
                decoded_images, _, q_loss = self.vqgan(imgs)

                disc_real = self.discriminator(imgs)
                disc_fake = self.discriminator(decoded_images)

                disc_factor = self.vqgan.adopt_weight(args.disc_factor, epoch*steps_per_epoch+i, threshold=args.disc_start)

                perceptual_loss = self.perceptual_loss(imgs, decoded_images)
                rec_loss = torch.abs(imgs - decoded_images)
                perceptual_rec_loss = args.perceptual_loss_factor * perceptual_loss + args.rec_loss_factor * rec_loss
                perceptual_rec_loss = perceptual_rec_loss.mean()
                g_loss = -torch.mean(disc_fake)

                λ = self.vqgan.calculate_lambda(perceptual_rec_loss, g_loss)
                vq_loss = perceptual_rec_loss + q_loss + disc_factor * λ * g_loss

                d_loss_real = torch.mean(F.relu(1. - disc_real))
                d_loss_fake = torch.mean(F.relu(1. + disc_fake))
                gan_loss = disc_factor * 0.5*(d_loss_real + d_loss_fake)

                self.opt_vq.zero_grad()
                vq_loss.backward(retain_graph=True)

                self.opt_disc.zero_grad()
                gan_loss.backward()

                self.opt_vq.step()
                self.opt_disc.step()

                if i % 400 == 0:
                    with torch.no_grad():
                        real_fake_images = torch.cat((imgs[:4], decoded_images.add(1).mul(0.5)[:4]))
                        vutils.save_image(real_fake_images, os.path.join(self.results_dir, f"{epoch}_{i}.jpg"), nrow=4)

                pbar.set_postfix(
                    VQ_Loss=np.round(vq_loss.cpu().detach().numpy().item(), 5),
                    GAN_Loss=np.round(gan_loss.cpu().detach().numpy().item(), 3)
                )
                pbar.update(0)
            torch.save(self.vqgan.state_dict(), os.path.join(self.checkpoint_dir, f"vqgan_epoch_{epoch}.pt"))





In [ ]:
class Generator:
  def __init__(self, args):
    self.work_dir = args.work_dir
    self.results_dir = os.path.join(self.work_dir, "results")
    self.checkpoint_dir = os.path.join(self.work_dir, "checkpoints")

    self.vqgan = VQGAN(args).to(device=torch.device(args.device))
    self.vqgan.load_checkpoint(os.path.join(self.checkpoint_dir, args.checkpoint))
    self.generate(args)

  @staticmethod
  def prepare_gen():
    os.makedirs(os.path.join(work_dir, "results"), exist_ok=True)

  def generate(self, args):
    inputs = load_data(args)
    with tqdm(range(len(inputs))) as pbar:
      for i, img in zip(pbar, inputs):
        img = img.to(device=args.device)
        result, _, _ = self.vqgan(img)
        vutils.save_image(result, os.path.join(self.results_dir, f"result_{i}.jpg"), nrow=4)
        pbar.update(0)


In [ ]:
class Arguments:
  def __init__(self, args):
    for k, v in args.items():
      setattr(self, k, v)

In [ ]:
dataset_path = './drive/MyDrive/VAE_learning/anime_faces/cropped/'
args_vqgan ={
    'mode': "gen",
    'checkpoint': "vqgan_epoch_0.pt",
    'latent_dim': 256,
    'image_size': 64,
    'num_codebook_vectors': 1024,
    'beta': 0.25,
    'image_channels': 3,
    'dataset_path': os.path.join(work_dir, "inputs"),
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'batch_size': 1,
    'epochs': 5,
    'learning_rate': 2.25e-5,
    'beta1': 0.5,
    'beta2': 0.9,
    'disc_start': 100,
    'disc_factor': 1,
    'rec_loss_factor': 1,
    'perceptual_loss_factor':1,
    'work_dir': work_dir
}
args_vqgan = Arguments(args_vqgan)

In [ ]:
if args_vqgan.mode == 'train':
  trainer = TrainVQGAN(args_vqgan)
else:
  generator = Generator(args_vqgan)

100%|██████████| 2/2 [00:16<00:00,  8.45s/it]
